## 공공데이터포털 API 연동

In [14]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import time
import calendar
import glob

In [ ]:
# API 설정
SERVICE_KEY = "a46bbaf106e41963c3883db630366f91e4960f1a08c831475517864811d806f4"
BASE_URL = "https://apis.data.go.kr/B552115/PvAmountByLocHr/getPvAmountByLocHr"

# 날짜 범위 설정
year = 2022
month = 12
last_day = calendar.monthrange(year, month)[1]

start_date = datetime(year, month, 1)
end_date = datetime(year, month, last_day)
num_of_rows = 1000

# 모든 데이터 수집
all_items = []
current_date = start_date

while current_date <= end_date:
    trade_ymd = current_date.strftime("%Y%m%d")
    
    # 첫 요청으로 해당 날짜의 totalCount 확인
    params = {
        "serviceKey": SERVICE_KEY,
        "pageNo": 1,
        "numOfRows": num_of_rows,
        "dataType": "json",
        "tradeYmd": trade_ymd
    }
    
    response = requests.get(BASE_URL, params=params)
    
    # 에러 처리
    if response.status_code != 200:
        print(f"에러 발생: {trade_ymd}, 상태코드: {response.status_code}")
        print(f"응답: {response.text[:200]}")
        break
    
    try:
        data = response.json()
    except:
        print(f"JSON 파싱 에러: {trade_ymd}")
        print(f"응답: {response.text[:200]}")
        break
    
    total_count = int(data["response"]["body"]["totalCount"])
    total_pages = (total_count // num_of_rows) + (1 if total_count % num_of_rows > 0 else 0)
    
    # 해당 날짜의 모든 페이지 수집
    for page in range(1, total_pages + 1):
        params["pageNo"] = page
        response = requests.get(BASE_URL, params=params)
        data = response.json()
        items = data["response"]["body"]["items"]["item"]
        all_items.extend(items)
        time.sleep(0.1)  # 0.1초 대기
    
    print(f"{trade_ymd} 완료: {total_count}개 데이터")
    current_date += timedelta(days=1)
    time.sleep(0.2)  # 날짜마다 0.2초 대기

# DataFrame 생성
df = pd.DataFrame(all_items)
print(f"\n총 {len(df)}개 데이터 수집 완료")
df.head()

In [ ]:
# rn 컬럼 제거 후 CSV 저장
df_save = df.drop(columns=["rn"])
SAVE_NAME = "data/2022/2022_12.csv"
df_save.to_csv(SAVE_NAME, index=False, encoding="utf-8-sig")
print(f"{SAVE_NAME} 저장 완료")

## 데이터 전처리

In [ ]:
# 발전량 데이터 전처리

# 전처리 이전 데이터 컬럼 정보
"""
tradeNo : 시간
tradeYmd : YYYYMMDD
regionNm : 지역 이름
amgo : 발전량(MWh)
"""

# 발전량 데이터 합치기
gen_files = glob.glob("data/*/gen*.csv")
df_gen = pd.concat([pd.read_csv(file) for file in gen_files], ignore_index=True)

# 서울시 데이터만 가져오기
df_gen = df_gen[df_gen["regionNm"] == "서울시"].reset_index(drop=True)

# 날짜 -> 시간순으로 정렬
df_gen = df_gen.sort_values(["tradeYmd", "tradeNo"], ascending=[True, True]).reset_index(drop=True)

# 지역 컬럼 제거
df_gen = df_gen.drop(columns=["regionNm"])

# 컬럼명 변경
df_gen.columns = ["time", "date", "generation"]

# 컬럼 순서 변경(날짜->시간->발전량)
df_gen = df_gen.iloc[:, [1,0,2]]

# 한시간 전 발전량 컬럼 추가(없으면 NaN)
df_gen["prev_generation"] = df_gen["generation"].shift(1)

# 어제 같은시간 발전량 컬럼 추가(없으면 NaN)
df_gen["prev_date"] = (pd.to_datetime(df_gen["date"].astype(str), format=("%Y%m%d")) - pd.Timedelta(days=1)).dt.strftime("%Y%m%d")

# prev_date 데이터 타입 date와 통일
df_gen["prev_date"] = df_gen["prev_date"].astype("int64")

# date와 prev_date를 비교해서 어제 발전량 저장
df_gen = df_gen.merge(df_gen[["date", "time", "generation"]], left_on=["prev_date", "time"], right_on=["date", "time"], how="left", suffixes=("", "_yesterday"))

# 불필요한 컬럼 제거
df_gen = df_gen.drop(columns=["prev_date", "date_yesterday"])

# 컬럼명 변경
df_gen = df_gen.rename(columns={"generation_yesterday": "yesterday_generation"})

df_gen.head(5)

In [ ]:
# 일사량 데이터 전처리

# 데이터 확인
# df.describe()
# df.info()

# 전처리 이전 데이터 컬럼 정보
"""
지점 : 지점번호
지점명 : 지점이름
일시 : YYYY-mm-dd HH:MM
일사(MJ/m2) : 일사량
"""

# 일사량 데이터 합치기
insolation_files = glob.glob("data/*/20*.csv")
df_insolation = pd.concat([pd.read_csv(file, encoding="cp949") for file in insolation_files], ignore_index=True)

# 불필요한 컬럼 정리
df_insolation = df_insolation[["일시", "일사(MJ/m2)"]]

# 일시 컬럼 date, time 으로 분리하기
df_insolation["datetime"] = pd.to_datetime(df_insolation["일시"])
df_insolation["date"] = df_insolation["datetime"].dt.strftime("%Y%m%d")
df_insolation["time"] = df_insolation["datetime"].dt.strftime("%H")

# 불필요한 컬럼 정리
df_insolation = df_insolation.drop(columns=["일시", "datetime"])

# 데이터 순서 정리
df_insolation = df_insolation[["date", "time", "일사(MJ/m2)"]]

# 컬럼명 변경
df_insolation = df_insolation.rename(columns={"일사(MJ/m2)": "insolation"})

# 데이터 타입 통일
df_insolation["date"] = df_insolation["date"].astype("int64")
df_insolation["time"] = df_insolation["time"].astype("int64")

df_insolation.head(20)

df_insolation.columns.to_list()

In [ ]:
# 발전량 데이터와 일사량 데이터 합치기

df_total = df_gen.merge(df_insolation[["date", "time", "insolation"]], on=["date", "time"], how="inner")

# NaN값 포함된 행 제거
df_total = df_total.dropna()

# df_total.to_csv("data/total_data.csv", index=False)

In [ ]:
# 최종 데이터 확인

df = pd.read_csv("data/total_data.csv")

# df.info()
# df.describe()
# df.isnull().sum()

# 추가 전처리

# 연도, 월, 일 분리
df["date"] = pd.to_datetime(df["date"].astype(str), format=("%Y%m%d"))
df["month"] = df["date"].dt.strftime("%m")
df["day"] = df["date"].dt.strftime("%d")
df["year"] = df["date"].dt.strftime("%Y")

# 컬럼 정리
df = df[["year", "month", "day", "time", "generation", "prev_generation", "yesterday_generation", "insolation"]]

df.dtypes

# 새로 저장
# df.to_csv("data/total_data.csv", index=False)

df.head(10)

dtype('int64')